In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [28]:
# collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.
mars_url = 'https://mars.nasa.gov/news/'
browser.visit(mars_url)
newshtml = browser.html
newsoup = bs(newshtml, 'html.parser')
time.sleep(3)
news_title = newsoup.find('div', class_='content_title').text.strip()
news_p = newsoup.find('div', class_='article_teaser_body').text.strip()
print(news_title)
print(news_p)

Curiosity Tastes First Sample in 'Clay-Bearing Unit'
This new region on Mars might reveal more about the role of water on Mount Sharp.


In [3]:
# find the image url for the current Featured Mars Image and assign the url string to a variable
jpl_url_search = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
jpl_url = 'https://www.jpl.nasa.gov'
browser.visit(jpl_url_search)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(1)
jplhtml = browser.html
jplsoup = bs(jplhtml, 'html.parser')
img_path = jplsoup.find('img', class_='fancybox-image')['src']
featured_image_url = jpl_url + img_path
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19334_ip.jpg


In [5]:
# scrape the latest Mars weather tweet
# twitter log in window pops up during initial run, should work second time running
marsweatherurl = 'https://twitter.com/marswxreport?lang=en'
browser.visit(marsweatherurl)
mars_html = browser.html
mars_soup = bs(mars_html, 'html.parser')
time.sleep(1)
mars_weather = mars_soup.find('p', class_='TweetTextSize')
print(mars_weather.text.strip().replace('hPapic', 'hPa pic'))

InSight sol 134 (2019-04-12) low -98.4ºC (-145.1ºF) high -18.2ºC (-0.8ºF)
pressure at 7.30 hPa pic.twitter.com/1DE7KRn8xy


In [42]:
# use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc
mars_facts_url = 'https://space-facts.com/mars/'
# facts_html = browser.html
# facts_soup = bs(facts_html, 'html.parser')
browser.visit(mars_facts_url)
table = pd.read_html(mars_facts_url)
df = table[0]
df.columns = ['description', 'value']
df = df.set_index('description')
mars_facts = df.to_html(index=True, header=True)

In [45]:
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [44]:
mars_facts

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>

In [23]:
# obtain high resolution images for each of Mar's hemispheres
hemisph_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
hlist = ['Cerberus Hemisphere', 'Schiaparelli Hemisphere', 'Syrtis Major Hemisphere', 'Valles Marineris Hemisphere']
hemisph_img_urls = []
for h in hlist:
    img_dict = {}
    browser.visit(hemisph_url)
    browser.click_link_by_partial_text(h)
    time.sleep(1)
    browser.find_link_by_text('Sample').first.click()
    
    browser.windows.current = browser.windows[-1]
    hemisph_soup = bs(browser.html, 'html.parser')
    img_url = hemisph_soup.find('img')['src']
    img_dict = {'title': h, 'img_url': img_url}
    hemisph_img_urls.append(img_dict)   
    browser.windows[-1].close()
    
    time.sleep(1)

In [24]:
hemisph_img_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]